In [18]:
import os
import re
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from google.colab import drive, userdata

# file management
drive.mount('/content/drive')
WORK_DIR = '/content/drive/MyDrive/Projects/skillextraction'

# work dir shortcut function
def work_dir(*args):
    return os.path.join(WORK_DIR, *args)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# load english sentence dataset from article
article_en = pd.read_csv(work_dir('Data', 'dataset.csv'))
article_en = article_en.groupby('skill').head(10)

# check
print(article_en.shape)
print(article_en['skill'].value_counts().value_counts())
article_en.head(3)

(138260, 2)
count
10    13826
Name: count, dtype: int64


,sentence,skill
0,the ideal candidate for this position should b...,advise customers on sewing patterns
1,we need an employee who is able to assist our ...,advise customers on sewing patterns
2,if you possess good communication skills and h...,advise customers on sewing patterns


In [20]:
# stitch up danish translations of dataset from article
article_da = pd.concat([pd.read_csv(work_dir('Translated_data', s)) for s in os.listdir(work_dir('Translated_data')) if re.match(r'^translated_sentences\_[0-9]+\.csv$', s)])
article_da = article_da.groupby('skill').head(10)

# check
print(article_da.shape)
print(article_da['skill'].value_counts().value_counts())
article_da.head(3)

(138170, 2)
count
10    13798
9        18
1         5
7         1
6         1
5         1
3         1
2         1
Name: count, dtype: int64


,skill,sentence
0,procurement legislation,en omfattende forståelse af indkøbslovgivning ...
1,procurement legislation,kendskab og erfaring med indkøbslovgivning er ...
2,procurement legislation,ansøgere med erfaring inden for områder relate...


In [21]:
# get replacements for leftovers
leftovers = pd.read_csv(work_dir('Translated_data', 'leftover_translated_sentences_0.csv'))

# check
print(leftovers.shape)
print(leftovers['skill'].value_counts().value_counts())
leftovers.head(3)

(280, 2)
count
10    28
Name: count, dtype: int64


,skill,sentence
0,Wireshark,ønske om at ansætte en erfaren person med erfa...
1,Wireshark,kun kandidater med en grundig forståelse af Wi...
2,Wireshark,Som netværkssikkerhedsanalytiker vil du anvend...


In [22]:
# combine main translations with leftover replacement (and ignore previous)
article_da = pd.concat([leftovers, article_da], ignore_index=True).groupby('skill').head(10)

# check
print(article_da.shape)
print(article_da['skill'].value_counts().value_counts())
article_da.head(3)

(138260, 2)
count
10    13826
Name: count, dtype: int64


,skill,sentence
0,Wireshark,ønske om at ansætte en erfaren person med erfa...
1,Wireshark,kun kandidater med en grundig forståelse af Wi...
2,Wireshark,Som netværkssikkerhedsanalytiker vil du anvend...


In [23]:
# stitch up extra danish dataset
extra_da = pd.concat([pd.read_csv(work_dir('Data', s)) for s in os.listdir(work_dir('Data')) if re.match(r'^sentences\_[0-9]+\.csv$', s)])
extra_da = extra_da.groupby('conceptUri').head(15)

# check
print(extra_da.shape)
print(extra_da['conceptUri'].value_counts().value_counts())
extra_da.head(3)

(208320, 2)
count
15    13885
14        2
9         1
8         1
Name: count, dtype: int64


,conceptUri,completion
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Vi søger en erfaren musikalsk leder til at til...
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,Som leder af musikalsk personale vil du være a...
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,Den ideelle kandidat vil have ekspertise inden...


In [24]:
# get esco english/danish
esco_en = pd.read_csv(work_dir('ESCO', 'ESCO dataset - v1.1.2 - classification - en - csv', 'skills_en.csv'))
esco_da = pd.read_csv(work_dir('ESCO', 'ESCO dataset - v1.1.2 - classification - da - csv', 'skills_da.csv'))

# limit to necessary and merge for easier mapping
esco = pd.merge(
    esco_en[['conceptUri', 'preferredLabel', 'description', 'altLabels']],
    esco_da[['conceptUri', 'preferredLabel', 'description', 'altLabels']],
    on='conceptUri',
    suffixes=('_en', '_da')
)

# check
print(esco.shape)
print(esco['preferredLabel_en'].value_counts().value_counts())
print(esco['preferredLabel_da'].value_counts().value_counts())
print(esco['description_en'].value_counts().value_counts())
print(esco['description_da'].value_counts().value_counts())
esco.head(3)

(13896, 7)
count
1    13896
Name: count, dtype: int64
count
1    13879
2        8
Name: count, dtype: int64
count
1    13890
2        3
Name: count, dtype: int64
count
1    13894
2        1
Name: count, dtype: int64


,conceptUri,preferredLabel_en,description_en,altLabels_en,preferredLabel_da,description_da,altLabels_da
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,Assign and manage staff tasks in areas such as...,manage staff of music\ncoordinate duties of mu...,lede musikalsk personale,Tildele og forvalte personaleopgaver på område...,NaN
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,Supervise the operations of a correctional fac...,oversee prison procedures\nmanage correctional...,føre tilsyn med fængselsprocedurer,Føre tilsyn med driften af et fængsel eller an...,NaN
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,"Identify oppression in societies, economies, c...",apply non-oppressive practices\napply an anti-...,anvende antioppressiv praksis,"Identificere undertrykkelse i samfund, økonomi...",NaN


In [25]:
# get rid of nans and duplicates (potential duplicate proxies!!!)
esco = esco.dropna(subset=['preferredLabel_da']) \
           .dropna(subset=['description_en']) \
           .dropna(subset=['description_da']) \
           .drop_duplicates(subset=['preferredLabel_da']) \
           .drop_duplicates(subset=['description_en']) \
           .drop_duplicates(subset=['description_da'])

# check
print(esco.shape)
print(esco['preferredLabel_en'].value_counts().value_counts())
print(esco['preferredLabel_da'].value_counts().value_counts())
print(esco['description_en'].value_counts().value_counts())
print(esco['description_da'].value_counts().value_counts())

(13884, 7)
count
1    13884
Name: count, dtype: int64
count
1    13884
Name: count, dtype: int64
count
1    13884
Name: count, dtype: int64
count
1    13884
Name: count, dtype: int64


In [26]:
# filter esco for what we have synthetic sentences (just ignore rest for now)
esco = esco[esco['preferredLabel_en'].isin(article_en['skill'])]

# check
print(esco.shape)
esco.head(3)

(13813, 7)


,conceptUri,preferredLabel_en,description_en,altLabels_en,preferredLabel_da,description_da,altLabels_da
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,Assign and manage staff tasks in areas such as...,manage staff of music\ncoordinate duties of mu...,lede musikalsk personale,Tildele og forvalte personaleopgaver på område...,NaN
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,Supervise the operations of a correctional fac...,oversee prison procedures\nmanage correctional...,føre tilsyn med fængselsprocedurer,Føre tilsyn med driften af et fængsel eller an...,NaN
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,"Identify oppression in societies, economies, c...",apply non-oppressive practices\napply an anti-...,anvende antioppressiv praksis,"Identificere undertrykkelse i samfund, økonomi...",NaN


In [27]:
# filter synthetic sentences for what we have esco! (less in danish?)
article_en = article_en[article_en['skill'].isin(esco['preferredLabel_en'])]
article_da = article_da[article_da['skill'].isin(esco['preferredLabel_en'])]
extra_da = extra_da[extra_da['conceptUri'].isin(esco['conceptUri'])]

# check
print(article_en.shape)
print(article_da.shape)
print(extra_da.shape)
print(article_en['skill'].value_counts().value_counts())
print(article_da['skill'].value_counts().value_counts())
print(extra_da['conceptUri'].value_counts().value_counts())
article_en.head(3)
article_da.head(3)
extra_da.head(3)

(138130, 2)
(138130, 2)
(207165, 2)
count
10    13813
Name: count, dtype: int64
count
10    13813
Name: count, dtype: int64
count
15    13808
14        2
9         1
8         1
Name: count, dtype: int64


,conceptUri,completion
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Vi søger en erfaren musikalsk leder til at til...
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,Som leder af musikalsk personale vil du være a...
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,Den ideelle kandidat vil have ekspertise inden...


In [28]:
# unfold the esco alt labels
alts_en = esco[['conceptUri', 'altLabels_en']].rename(columns={'altLabels_en': 'sentence'}) \
                                              .set_index(['conceptUri'])['sentence'] \
                                              .str.split('\n') \
                                              .explode() \
                                              .reset_index() \
                                              .dropna()

alts_da = esco[['conceptUri', 'altLabels_da']].rename(columns={'altLabels_da': 'sentence'}) \
                                              .set_index(['conceptUri'])['sentence'] \
                                              .str.split('\n') \
                                              .explode() \
                                              .reset_index() \
                                              .dropna()

# check
print(alts_en['conceptUri'].value_counts().value_counts())
print(alts_da['conceptUri'].value_counts().value_counts())
alts_en

count
5     2049
6     2045
8     1993
7     1937
9     1267
4      787
10     705
11     613
12     581
1      536
3      368
2      243
13     237
14      97
15      61
16      29
17      21
18      16
20      11
19       7
22       3
32       3
24       2
46       1
21       1
25       1
26       1
30       1
35       1
38       1
40       1
45       1
56       1
Name: count, dtype: int64
count
1     314
5     187
4     123
3      87
6      83
2      55
7      28
8       6
9       2
17      1
13      1
10      1
Name: count, dtype: int64


,conceptUri,sentence
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage staff of music
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,coordinate duties of musical staff
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage music staff
3,http://data.europa.eu/esco/skill/0005c151-5b5a...,direct musical staff
4,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff
...,...,...
97035,http://data.europa.eu/esco/skill/fff74a70-4f82...,position guardrails and toeboards
97036,http://data.europa.eu/esco/skill/fff74a70-4f82...,fit guardrails and toeboards
97037,http://data.europa.eu/esco/skill/fff74a70-4f82...,fitting of guardrails and toeboards
97038,http://data.europa.eu/esco/skill/fff74a70-4f82...,guardrail and toeboard attachment


In [29]:
# stitch up danish translations of english alt labels
alts_en_trans = pd.concat([pd.read_csv(work_dir('Translated_data', s)) for s in os.listdir(work_dir('Translated_data')) if re.match(r'^translated\_alts\_[0-9]+\.csv$', s)])
alts_en_trans = alts_en_trans[alts_en_trans['conceptUri'].isin(esco['conceptUri'])].dropna()

# check
print(alts_en_trans.shape)
print(alts_en_trans['conceptUri'].value_counts().value_counts())
alts_en_trans

(92722, 2)
count
10    2389
5     2050
6     2043
8     1993
7     1935
9     1267
4      788
1      535
3      368
2      242
16       3
11       3
12       2
20       1
18       1
14       1
Name: count, dtype: int64


,conceptUri,sentence
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,lede musikpersonale
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,koordinere opgaver for musikpersonale
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,lede musikstaben
3,http://data.europa.eu/esco/skill/0005c151-5b5a...,instrukt musikpersonalet
4,http://data.europa.eu/esco/skill/0005c151-5b5a...,lede det musikalske personale
...,...,...
24,http://data.europa.eu/esco/skill/fff74a70-4f82...,fiksering af rækværk og fodlister
25,http://data.europa.eu/esco/skill/fff74a70-4f82...,placere rækværk og fodlister
26,http://data.europa.eu/esco/skill/fff74a70-4f82...,montere rækværk og fodlister
27,http://data.europa.eu/esco/skill/fff74a70-4f82...,montering af rækværk og fodlister


In [30]:
# load generated danish multi skill sentences
multi = pd.read_json(work_dir('Data', 'multiskills.json'), orient='records', lines=True)
multi_prim = multi[['conceptUriPrimary', 'sentence']].rename(columns={'conceptUriPrimary': 'conceptUri'})
multi_sec = multi[['conceptUriSecondary', 'sentence']].rename(columns={'conceptUriSecondary': 'conceptUri'})

# filter
multi_prim = multi_prim[multi_prim['conceptUri'].isin(esco['conceptUri'])].dropna()
multi_sec = multi_sec[multi_sec['conceptUri'].isin(esco['conceptUri'])].dropna()

# check
print(multi_prim.shape)
print(multi_sec.shape)
print(multi_prim['conceptUri'].value_counts().value_counts())
print(multi_sec['conceptUri'].value_counts().value_counts())
multi_prim
multi_sec

(129690, 2)
(129851, 2)
count
10    12969
Name: count, dtype: int64
count
4      989
5      933
7      917
3      895
6      895
      ... 
73       1
69       1
64       1
62       1
156      1
Name: count, Length: 81, dtype: int64


,conceptUri,sentence
0,http://data.europa.eu/esco/skill/4109c79f-0332...,Vi søger en leder til at styre musikalsk perso...
1,http://data.europa.eu/esco/skill/339f165c-0002...,Som koordinator skal du lede musikalsk persona...
2,http://data.europa.eu/esco/skill/aa755e1d-81cd...,"Du skal lede musikalsk personale, mens du posi..."
3,http://data.europa.eu/esco/skill/3e2d3720-84e3...,I denne rolle vil du lede musikalsk personale ...
4,http://data.europa.eu/esco/skill/fbb9ceec-26c2...,Som musikchef er det vigtigt at lede musikalsk...
...,...,...
130395,http://data.europa.eu/esco/skill/91991706-cd8e...,"Du skal kunne planlægge stilladser, hvilket er..."
130396,http://data.europa.eu/esco/skill/3ae709bc-5c3b...,"For at kunne anbringe gelændere og fodbrædder,..."
130397,http://data.europa.eu/esco/skill/e3d02758-7621...,Det er nødvendigt at placere udriggere korrekt...
130398,http://data.europa.eu/esco/skill/c5879759-c23a...,En vigtig del af jobbet er at installere hejse...


In [31]:
# map label to conceptUri
map = dict(zip(esco['preferredLabel_en'], esco['conceptUri']))

# combine it all
skills = pd.concat([
    esco[['conceptUri', 'preferredLabel_en']].rename(columns={'preferredLabel_en': 'sentence'}).assign(group=1),
    esco[['conceptUri', 'preferredLabel_da']].rename(columns={'preferredLabel_da': 'sentence'}).assign(group=2),
    esco[['conceptUri', 'description_en']].rename(columns={'description_en': 'sentence'}).assign(group=3),
    esco[['conceptUri', 'description_da']].rename(columns={'description_da': 'sentence'}).assign(group=4),
    article_en[['sentence']].assign(conceptUri=article_en['skill'].map(map)).assign(group=5),
    article_da[['sentence']].assign(conceptUri=article_da['skill'].map(map)).assign(group=6),
    extra_da[['conceptUri', 'completion']].rename(columns={'completion': 'sentence'}).assign(group=7), # very slightly unbalanced
    alts_en[['conceptUri', 'sentence']].assign(group=8), # unbalanced
    alts_en_trans[['conceptUri', 'sentence']].assign(group=9), # unbalanced
    alts_da[['conceptUri', 'sentence']].assign(group=9), # unbalanced (group together alts_en_trans and alts_da due to so few in alts_da)
    multi_prim[['conceptUri', 'sentence']].assign(group=10),
    multi_sec[['conceptUri', 'sentence']].assign(group=11) # unbalanced
])

# check
print(skills.shape)
print(skills['conceptUri'].value_counts().value_counts())
skills.sample(10)

(990700, 3)
count
72     677
69     668
70     633
68     627
71     606
      ... 
139      1
137      1
132      1
130      1
217      1
Name: count, Length: 107, dtype: int64


,conceptUri,sentence,group
13901,http://data.europa.eu/esco/skill/45f3857b-072f...,Den ideelle kandidat vil have erfaring med at ...,6
136671,http://data.europa.eu/esco/skill/bf1b6671-e8f4...,a passion for fashion from other cultures and ...,5
26873,http://data.europa.eu/esco/skill/46f2f6b5-3112...,prepare veterinary surgical environment,8
22539,http://data.europa.eu/esco/skill/29ac0064-fcd6...,Kandidater skal være i stand til at demonstrer...,6
7325,http://data.europa.eu/esco/skill/d8045ecd-22f3...,Ved at betjene præcisionsmaskineri og udvikle ...,11
5517,http://data.europa.eu/esco/skill/3de6d427-546b...,Det er afgørende at tilstræbe bevaring af vand...,11
92372,http://data.europa.eu/esco/skill/f40da95c-febe...,modify different audio-visual formats,8
903,http://data.europa.eu/esco/skill/ca1a708c-0178...,Ønsker du at arbejde med at sikre høj kvalitet...,7
96447,http://data.europa.eu/esco/skill/307f11d5-1356...,i stand til at arbejde i et hurtigt tempo med ...,6
9583,http://data.europa.eu/esco/skill/135192cb-84e4...,En vigtig opgave vil være at analysere lærings...,10


In [32]:
# save!
skills.to_json(work_dir('Data', 'skills.json'), orient='records', lines=True, index=False)

In [33]:
# sanity check
df = pd.read_json(work_dir('Data', 'skills.json'), orient='records', lines=True)
print(df.shape)
df.head(3)

(990700, 3)


,conceptUri,sentence,group
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,1
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,1
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,1
